# Gurobi multiple ml models
Update previos notebook with the delta constraints in decision variables. As this problem try to represent an Industrial Process, physically there are some limitations in the dinamics of the process and the values of the decision process can't change too fast.

**In this update adding a constraints to limite the rate of change of the decision variables (only the decision variables that are features of machine learning models, not the decision variables that are targets)**


-------
The constraints added are the follow:

si quiero calcular el valor absoluto de una diferencia de variables de decisión se tiene la **variable auxiliar y** para representar el **valor absoluto de x1-x2**

- y >= (x1 - x2)

- y >= -(x1 - x2)

- y <= delta

## Root folder and read env variables

In [1]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## RUN

In [2]:
import pickle
import pandas as pd
import numpy as np

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

## USER INPUT
User input:
- Actual values features no controlables
- Actual values features controlables
- Delta permitido features controlables

In [3]:
########## d0eop_microkappa

############################### Actual values features no controlables ###############################
input_kappa_d0 = 6.349346  # "240AIT063A.PNT"
input_brillo_d0 = 61.826925 # "240AIT063B.PNT"
input_calc_prod_d0 = 3346.85825 # "calc_prod_d0"
input_dqo_evaporadores = 707.265937 # "SSTRIPPING015"
input_concentracion_clo2_d0 = 11.516543 #"S276PER002"
input_ph_a = 2.93846 # "240AIC022.MEAS"


############################### Actual values features controlables ###############################
input_actual_value_especifico_dioxido_d0 = 6.46 #"240FY050.RO02"
input_actual_value_especifico_oxigeno_eop = 1.08 #"240FY118B.RO01"
input_actual_value_especifico_peroxido_eop = 4.90 #"240FY11PB.RO01"
input_actual_value_especifico_soda_eop = 8.84 #"240FY107A.RO01"


############################### Delta permitido features controlables ############################### (it is neccesary the values be float to change decimals delta)
input_delta_especifico_dioxido_d0 = 0.5
input_delta_especifico_oxigeno_eop = 1.0
input_delta_especifico_peroxido_eop = 1.0
input_delta_especifico_soda_eop = 1.0

In [4]:
########## d1_brillo

############################### Actual values features no controlables ###############################
input_blancura_d1 = 85.051634 #"240AIT225B.PNT"
input_prod_bypass = 169.45041 #"240FI108A.PNT"
input_calc_prod_d1 = 3347.664057 #"calc_prod_d1"
input_temperatura_d1 = 77.212822 #"240TIT223.PNT"


############################### Actual values features controlables ###############################
input_actual_value_especifico_dioxido_d1 = 2.89 #"240FY218.RO02"
input_actual_value_especifico_acido_d1 = 1.93 #"240FY210A.RO01"


############################### Delta permitido features controlables ###############################
input_delta_especifico_dioxido_d1 = 1.0
input_delta_especifico_acido_d1 = 1.0

In [5]:
########## p_blancura

############################### Actual values features no controlables ###############################
input_calc_prod_p = 3346.185561 #"calc_prod_p"
input_ph_p = 10.382279 #"240AIC324.MEAS"


############################### Actual values features controlables ###############################
input_actual_value_especifico_soda_p = 2.65 #"240FY312.RO01"
input_actual_value_especifico_peroxido_p = 0.78 #"240FY397.RO01"
input_actual_value_especifico_acido_p = 2.01 #"240FY430.RO01"


############################### Delta permitido features controlables ###############################
input_delta_especifico_soda_p = 1.0
input_delta_especifico_peroxido_p = 1.0
input_delta_especifico_acido_p = 1.0

## PREPARATION

### 2. Load model machine learning - MANUALLY - HARDCODED

In [6]:
######################## read model d0eop_microkappa ########################
path_model_d0eop_microkappa = f'models/d0eop_microkappa/lr.pkl'
model_d0eop_microkappa = pd.read_pickle(path_model_d0eop_microkappa)
model_d0eop_microkappa

Pipeline(steps=[('poly_feature_2', PolynomialFeatures()),
                ('lr', LinearRegression())])

In [7]:
######################## read model d1_brillo ########################
path_model_d1_brillo = f'models/d1_brillo/lr.pkl'
model_d1_brillo = pd.read_pickle(path_model_d1_brillo)
model_d1_brillo

LinearRegression()

In [8]:
######################## read model p_blancura ########################
path_model_p_blancura = f'models/p_blancura/lr.pkl'
model_p_blancura = pd.read_pickle(path_model_p_blancura)
model_p_blancura

Pipeline(steps=[('poly_feature_2', PolynomialFeatures()),
                ('lr', LinearRegression())])

### 3. Define list of features and target for each model

In [9]:
######################## model d0eop_microkappa ########################

list_features_d0eop_microkappa = [
    "240AIT063A.PNT", #kappa_d0
    "240AIT063B.PNT", #brillo_d0
    "calc_prod_d0", #calc_prod_d0
    "240FY050.RO02", #especifico_dioxido_d0 - VC
    "SSTRIPPING015", #dqo_evaporadores
    "S276PER002", #concentracion_clo2_d0
    "240AIC022.MEAS", #ph_a
    "240FY118B.RO01", #especifico_oxigeno_eop - VC
    "240FY11PB.RO01", #especifico_peroxido_eop - VC
    "240FY107A.RO01", #especifico_soda_eop - VC
]

list_features_controlables_d0eop_microkappa = [
    "240FY050.RO02", #especifico_dioxido_d0 - VC,
    "240FY118B.RO01", #especifico_oxigeno_eop - VC
    "240FY11PB.RO01", #especifico_peroxido_eop - VC
    "240FY107A.RO01", #especifico_soda_eop - VC
]

list_target_d0eop_microkappa = ['240AIT225A.PNT'] # microkappa_d1

In [10]:
######################## model d1_brillo ########################

list_features_d1_brillo = [
    "240AIT225B.PNT", # blancura_d1
    "240AIT225A.PNT", # microkappa_d1
    "240FI108A.PNT", # prod_bypass
    "calc_prod_d1", #calc_prod_d1
    "240FY218.RO02", #especifico_dioxido_d1 - VC
    "240TIT223.PNT", #temperatura_d1
    "240FY210A.RO01", # especifico_acido_d1 - vc
]

list_features_controlables_d1_brillo = [
    "240FY218.RO02", #especifico_dioxido_d1 - VC
    "240FY210A.RO01", # especifico_acido_d1 - vc
]

list_target_d1_brillo = ['240AIT322B.PNT'] # brillo_entrada_p

In [11]:
######################## model p_blancura ########################

list_features_p_blancura = [
    "240AIT322B.PNT", #brillo_p
    "calc_prod_p", #calc_prod_p
    "240FY312.RO01", # especifico_soda_p - VC
    "240FY397.RO01", #especifico_peroxido_p - VC
    "240AIC324.MEAS", #ph_p,
    "240FY430.RO01", #especifico_acido_p - VC
]

list_features_controlables_p_blancura = [    
    "240FY312.RO01", # especifico_soda_p - VC
    "240FY397.RO01", #especifico_peroxido_p - VC
    "240FY430.RO01", #especifico_acido_p - VC
]

list_target_p_blancura = ['240AIT416B.PNT'] #blancura_salida_p

### 4. Load tables parameters for optimization - bounds

In [12]:
# load bounds of decision variables in optimization that also are FEATURES in machine learning models

path_bounds_decision_var_features = 'config/Optimization-Bounds-DecisionVar.xlsx'
bounds_decision_var_features = pd.read_excel(path_bounds_decision_var_features)
bounds_decision_var_features

,MODEL,TAG,TAG_DESCRIPTION,DESCRIPCION,MIN_VALUE,MAX_VALUE
0,d0eop_microkappa,240FY050.RO02,especifico_dioxido_d0,Específico ClO2,2,11
1,d0eop_microkappa,240FY118B.RO01,especifico_oxigeno_eop,Esp. Oxígeno,0,2
2,d0eop_microkappa,240FY11PB.RO01,especifico_peroxido_eop,Esp. Peróxido,1,15
3,d0eop_microkappa,240FY107A.RO01,especifico_soda_eop,Esp. Soda EOP,4,20
4,d1_brillo,240FY218.RO02,especifico_dioxido_d1,Esp Dióxido,1,10
5,d1_brillo,240FY210A.RO01,especifico_acido_d1,Esp Acido,0,10
6,p_blancura,240FY312.RO01,especifico_soda_p,Esp Soda,0,10
7,p_blancura,240FY397.RO01,especifico_peroxido_p,Esp Peróxido P,0,10
8,p_blancura,240FY430.RO01,especifico_acido_p,Esp Acido tac blanca,0,10


In [13]:
# load bounds of decision variables in optimization that also are TARGETS in machine learning models
# obs: the targets of one model are features in the next model. They are defined once

path_bounds_decison_var_target = 'config/Optimization-Bounds-Target.xlsx'
bounds_decison_var_target = pd.read_excel(path_bounds_decison_var_target)
bounds_decison_var_target

,MODEL,TAG,TAG_DESCRIPTION,DESCRIPCION,MIN_VALUE,MAX_VALUE
0,d0eop_microkappa,240AIT225A.PNT,microkappa_d1,mKappa salida EOP (entrada D1 ),1.9,2.4
1,d1_brillo,240AIT322B.PNT,brillo_entrada_p,Brillo salida D1 (entrada P),89.2,100.0
2,p_blancura,240AIT416B.PNT,blancura_salida_p,Blancura Salida P linea,90.0,91.0


### 5. Load Prices
Parameters in Optimization

In [14]:
path_prices = 'config/price-chemicals.xlsx'
prices = pd.read_excel(path_prices)
prices

,acido,peroxido,soda,oxigeno,dioxido
0,0.275,0.698,0.655,0.092,1.4


## RUN OPTIMIZATION

### 0. Load transversal params - sets of optimization model
Transversal all codes, not only this code. For example order in features in the data.

Save the sets of optimization model as pandas index

In [15]:
# define set to models. In this example there is a set with only one element
list_bleaching = ['bleaching']
index_bleaching = pd.Index(list_bleaching)
index_bleaching

Index(['bleaching'], dtype='object')

### 1. Create guroby optimization model
Documentation: https://www.gurobi.com/documentation/current/refman/py_model.html

In [16]:
# env = gp.Env(params=params)

#Create the model within the Gurobi environment
m = gp.Model(name = "Bleaching Optimization v2")

Restricted license - for non-production use only - expires 2025-11-24


### 2. Upper bounds and lower bounds of decision variables

### 3. Input parameters of optimization model
- Actual values of decision variables and delta rate of change (Get the actual values of decision variables that are features of machine learning models and define a rate of change of these variables)

#### 3.1 Actual values of decision variables

Get the actual values of decision variables that are features of machine learning models

In [17]:
######################## actual values for model d0eop_microkappa ########################
### IMPORTANTE ASEGURSE QUE EL TAG CORRESPONDE CON EL NOMBRE DEFINIDO


#especifico_dioxido_d0
actual_value_especifico_dioxido_d0 = input_actual_value_especifico_dioxido_d0

#especifico_oxigeno_eop
actual_value_especifico_oxigeno_eop = input_actual_value_especifico_oxigeno_eop

#especifico_peroxido_eop
actual_value_especifico_peroxido_eop = input_actual_value_especifico_peroxido_eop

#especifico_soda_eop
actual_value_especifico_soda_eop = input_actual_value_especifico_soda_eop

In [18]:
######################## actual values for model d1_brillo ########################

#especifico_dioxido_d1
actual_value_especifico_dioxido_d1 = input_actual_value_especifico_dioxido_d1

#especifico_acido_d1
actual_value_especifico_acido_d1 = input_actual_value_especifico_acido_d1

In [19]:
######################## actual values for model p_blancura ########################
list_features_controlables_p_blancura


#especifico_soda_p
actual_value_especifico_soda_p = input_actual_value_especifico_soda_p


#especifico_peroxido_p
actual_value_especifico_peroxido_p = input_actual_value_especifico_peroxido_p


#especifico_acido_p
actual_value_especifico_acido_p = input_actual_value_especifico_acido_p

#### 3.2 Parameters Rate of Change decision variables
TODO: IMPORTANT. THIS SHOULD BE DEFINED IN A EXCEL AS A CONFIG FILE. but for now, is hardocoded

In [20]:
######################## actual values for model d0eop_microkappa ########################

#especifico_dioxido_d0
delta_especifico_dioxido_d0 = input_delta_especifico_dioxido_d0

#especifico_oxigeno_eop
delta_especifico_oxigeno_eop = input_delta_especifico_oxigeno_eop

#especifico_peroxido_eop
delta_especifico_peroxido_eop = input_delta_especifico_peroxido_eop

#especifico_soda_eop
delta_especifico_soda_eop = input_delta_especifico_soda_eop

In [21]:
######################## actual values for model d1_brillo ########################

#especifico_dioxido_d1
delta_especifico_dioxido_d1 = input_delta_especifico_dioxido_d1

#especifico_acido_d1
delta_especifico_acido_d1 = input_delta_especifico_acido_d1

In [22]:
######################## actual values for model p_blancura ########################

#especifico_soda_p
delta_especifico_soda_p = input_delta_especifico_soda_p


#especifico_peroxido_p
delta_especifico_peroxido_p = input_delta_especifico_peroxido_p

#especifico_acido_p
delta_especifico_acido_p = input_delta_especifico_acido_p

In [23]:
# price parameters
prices

,acido,peroxido,soda,oxigeno,dioxido
0,0.275,0.698,0.655,0.092,1.4


### 4. Features input machine learning model fixed (that are not decision variables or parameters in optimization model)
Define the features that are inputs of machine learning model that are not decision variables of optimization model (so this values doesn't change). And also, this features that are not parameters of optimization model, so this values are not used in the restrictions

**Generar una instancia a partir de los datos para las feautures de los modelos de ML que no son variables de decisión ni targets. La instancia generada se hace a partir de los datos. Para este ejemplo se va a considerar la media**

In [24]:
######################## generate instance for model d0eop_microkappa ########################

# list feature NC
list_features_d0eop_microkappa_no_vc = list(set(list_features_d0eop_microkappa) - set(list_features_controlables_d0eop_microkappa))

# generate dataframe with the mean
#instance_no_controlables_d0eop_microkappa = data[list_features_d0eop_microkappa_no_vc].mean().to_frame().T
instance_no_controlables_d0eop_microkappa = pd.DataFrame()
instance_no_controlables_d0eop_microkappa["240AIT063A.PNT"] = [input_kappa_d0]
instance_no_controlables_d0eop_microkappa["240AIT063B.PNT"] = [input_brillo_d0]
instance_no_controlables_d0eop_microkappa["calc_prod_d0"] = [input_calc_prod_d0]
instance_no_controlables_d0eop_microkappa["SSTRIPPING015"] = [input_dqo_evaporadores]
instance_no_controlables_d0eop_microkappa["S276PER002"] = [input_concentracion_clo2_d0]
instance_no_controlables_d0eop_microkappa["240AIC022.MEAS"] = [input_ph_a]

instance_no_controlables_d0eop_microkappa

,240AIT063A.PNT,240AIT063B.PNT,calc_prod_d0,SSTRIPPING015,S276PER002,240AIC022.MEAS
0,6.349346,61.826925,3346.85825,707.265937,11.516543,2.93846


In [25]:
######################## generate instance for model d1_brillo ########################

# list features NC
list_features_d1_brillo_no_vc = list(set(list_features_d1_brillo) - set(list_features_controlables_d1_brillo)) # substract vc d1_brillo
list_features_d1_brillo_no_vc = list(set(list_features_d1_brillo_no_vc) - set(list_target_d0eop_microkappa))# substract target d0eop_microkappa

# generate dataframe with the mean
#instance_no_controlables_d1_brillo = data[list_features_d1_brillo_no_vc].mean().to_frame().T
instance_no_controlables_d1_brillo = pd.DataFrame()
instance_no_controlables_d1_brillo["240AIT225B.PNT"] = [input_blancura_d1]
instance_no_controlables_d1_brillo["240FI108A.PNT"] = [input_prod_bypass]
instance_no_controlables_d1_brillo["calc_prod_d1"] = [input_calc_prod_d1]
instance_no_controlables_d1_brillo["240TIT223.PNT"] = [input_temperatura_d1]

instance_no_controlables_d1_brillo

,240AIT225B.PNT,240FI108A.PNT,calc_prod_d1,240TIT223.PNT
0,85.051634,169.45041,3347.664057,77.212822


In [26]:
######################## generate instance for model p_blancura ########################

# list features NC
list_features_p_blancura_no_vc = list(set(list_features_p_blancura) - set(list_features_controlables_p_blancura)) # substract vc p_blancura
list_features_p_blancura_no_vc = list(set(list_features_p_blancura_no_vc) - set(list_target_d1_brillo))# substract target d1_brillo

# generate dataframe with the mean
#instance_no_controlables_p_blancura = data[list_features_p_blancura_no_vc].mean().to_frame().T
instance_no_controlables_p_blancura = pd.DataFrame()
instance_no_controlables_p_blancura["calc_prod_p"] = [input_calc_prod_p]
instance_no_controlables_p_blancura["240AIC324.MEAS"] = [input_ph_p]

instance_no_controlables_p_blancura

,calc_prod_p,240AIC324.MEAS
0,3346.185561,10.382279


IMPORTANT: En los modelos de las etapas siguientes se deben borrar las features que son variables controlables (variables de decisión optimizador) y además los target de etapas previas que se vuelven variables no controlables de la etapa siguiente (ya que también son variables de decisión del optimizador)

### 5. Decision variables of optimization model

Let us now define the decision variables. In our model, we want to store the price and number of avocados allocated to each region. We also want variables that track how many avocados are predicted to be sold and how many are predicted to be wasted. 

All those variables are created using gurobipy-pandas, with the function `gppd.add_vars`. To use this function it is necessary to define:
- model: optimization model of gurobi
- index: pandas index. With this index it can defined the sets of the decision variables
- name: name of the decision variable
- Example: x = gppd.add_vars(model, index, name="x")

In [27]:
######################## decision variables that are FEATURES in Machiine Learning Models ########################

# model d0eop_microkappa
especifico_dioxido_d0 = gppd.add_vars(m, index_bleaching, name = "especifico_dioxido_d0", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_dioxido_d0']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_dioxido_d0']['MAX_VALUE'].values[0]
                                     )

especifico_oxigeno_eop = gppd.add_vars(m, index_bleaching, name = "especifico_oxigeno_eop", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_oxigeno_eop']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_oxigeno_eop']['MAX_VALUE'].values[0]
                                     )

especifico_peroxido_eop = gppd.add_vars(m, index_bleaching, name = "especifico_peroxido_eop", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_peroxido_eop']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_peroxido_eop']['MAX_VALUE'].values[0]
                                     )

especifico_soda_eop = gppd.add_vars(m, index_bleaching, name = "especifico_soda_eop", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_soda_eop']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_soda_eop']['MAX_VALUE'].values[0]
                                     )



# model d1_brillo
especifico_dioxido_d1 = gppd.add_vars(m, index_bleaching, name = "especifico_dioxido_d1", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_dioxido_d1']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_dioxido_d1']['MAX_VALUE'].values[0]
                                     )

especifico_acido_d1 = gppd.add_vars(m, index_bleaching, name = "especifico_acido_d1", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_acido_d1']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_acido_d1']['MAX_VALUE'].values[0]
                                     )



# model p_blancura
especifico_soda_p = gppd.add_vars(m, index_bleaching, name = "especifico_soda_p", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_soda_p']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_soda_p']['MAX_VALUE'].values[0]
                                     )

especifico_peroxido_p = gppd.add_vars(m, index_bleaching, name = "especifico_peroxido_p", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_peroxido_p']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_peroxido_p']['MAX_VALUE'].values[0]
                                     )

especifico_acido_p = gppd.add_vars(m, index_bleaching, name = "especifico_acido_p", 
                                      lb = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_acido_p']['MIN_VALUE'].values[0], 
                                      ub = bounds_decision_var_features[bounds_decision_var_features['TAG_DESCRIPTION'] == 'especifico_acido_p']['MAX_VALUE'].values[0]
                                     )

In [28]:
######################## decision variables that are TARGETS in Machiine Learning Models ########################

# model d0eop_microkappa
microkappa_d1 = gppd.add_vars(m, index_bleaching, name = "especifico_dioxido_d0", 
                                      lb = bounds_decison_var_target[bounds_decison_var_target['TAG_DESCRIPTION'] == 'microkappa_d1']['MIN_VALUE'].values[0], 
                                      ub = bounds_decison_var_target[bounds_decison_var_target['TAG_DESCRIPTION'] == 'microkappa_d1']['MAX_VALUE'].values[0]
                                     )

# model d1_brillo
brillo_entrada_p = gppd.add_vars(m, index_bleaching, name = "especifico_dioxido_d0", 
                                      lb = bounds_decison_var_target[bounds_decison_var_target['TAG_DESCRIPTION'] == 'brillo_entrada_p']['MIN_VALUE'].values[0], 
                                      ub = bounds_decison_var_target[bounds_decison_var_target['TAG_DESCRIPTION'] == 'brillo_entrada_p']['MAX_VALUE'].values[0]
                                     )
# model p_blancura
blancura_salida_p = gppd.add_vars(m, index_bleaching, name = "especifico_dioxido_d0", 
                                      lb = bounds_decison_var_target[bounds_decison_var_target['TAG_DESCRIPTION'] == 'blancura_salida_p']['MIN_VALUE'].values[0], 
                                      ub = bounds_decison_var_target[bounds_decison_var_target['TAG_DESCRIPTION'] == 'blancura_salida_p']['MAX_VALUE'].values[0]
                                     )

In [29]:
############ decision variables that represent the difference between actual value FEATURES in Machine Learning Models and optimal value ############


# model d0eop_microkappa
diff_especifico_dioxido_d0 = gppd.add_vars(m, index_bleaching, name = "diff_especifico_dioxido_d0", 
                                      lb = -gp.GRB.INFINITY,
                                      ub = gp.GRB.INFINITY
                                     )

diff_especifico_oxigeno_eop = gppd.add_vars(m, index_bleaching, name = "diff_especifico_oxigeno_eop", 
                                      lb = -gp.GRB.INFINITY,
                                      ub = gp.GRB.INFINITY
                                     )

diff_especifico_peroxido_eop = gppd.add_vars(m, index_bleaching, name = "diff_especifico_peroxido_eop", 
                                      lb = -gp.GRB.INFINITY,
                                      ub = gp.GRB.INFINITY
                                     )

diff_especifico_soda_eop = gppd.add_vars(m, index_bleaching, name = "diff_especifico_soda_eop", 
                                      lb = -gp.GRB.INFINITY,
                                      ub = gp.GRB.INFINITY
                                     )



# model d1_brillo
diff_especifico_dioxido_d1 = gppd.add_vars(m, index_bleaching, name = "diff_especifico_dioxido_d1", 
                                      lb = -gp.GRB.INFINITY,
                                      ub = gp.GRB.INFINITY
                                     )

diff_especifico_acido_d1 = gppd.add_vars(m, index_bleaching, name = "diff_especifico_acido_d1", 
                                      lb = -gp.GRB.INFINITY,
                                      ub = gp.GRB.INFINITY
                                     )



# model p_blancura
diff_especifico_soda_p = gppd.add_vars(m, index_bleaching, name = "diff_especifico_soda_p", 
                                      lb = -gp.GRB.INFINITY,
                                      ub = gp.GRB.INFINITY
                                     )

diff_especifico_peroxido_p = gppd.add_vars(m, index_bleaching, name = "diff_especifico_peroxido_p", 
                                      lb = -gp.GRB.INFINITY,
                                      ub = gp.GRB.INFINITY
                                     )

diff_especifico_acido_p = gppd.add_vars(m, index_bleaching, name = "diff_especifico_acido_p", 
                                      lb = -gp.GRB.INFINITY,
                                      ub = gp.GRB.INFINITY
                                     )

### 6. Constraints (constraints that are not generated by a ml model)
Nota: para que tomen las restricciones es necesario pasar el arg [0] de la variable de decisión

In [30]:
######################## decision variables that are FEATURES in Machiine Learning Models ########################

###### model d0eop_microkappa ######
# especifico_dioxido_d0
m.addConstr(diff_especifico_dioxido_d0[0] >= (especifico_dioxido_d0[0] - actual_value_especifico_dioxido_d0), name = 'diff_especifico_dioxido_d0 positive segment')
m.addConstr(diff_especifico_dioxido_d0[0] >= -(especifico_dioxido_d0[0] - actual_value_especifico_dioxido_d0), name = 'diff_especifico_dioxido_d0 negative segment')
m.addConstr(diff_especifico_dioxido_d0[0] <= delta_especifico_dioxido_d0, name = 'diff_especifico_dioxido_d0 delta')


# especifico_oxigeno_eop
m.addConstr(diff_especifico_oxigeno_eop[0] >= (especifico_oxigeno_eop[0] - actual_value_especifico_oxigeno_eop), name = 'diff_especifico_oxigeno_eop positive segment')
m.addConstr(diff_especifico_oxigeno_eop[0] >= -(especifico_oxigeno_eop[0] - actual_value_especifico_oxigeno_eop), name = 'diff_especifico_oxigeno_eop negative segment')
m.addConstr(diff_especifico_oxigeno_eop[0] <= delta_especifico_oxigeno_eop, name = 'diff_especifico_oxigeno_eop delta')


# especifico_peroxido_eop
m.addConstr(diff_especifico_peroxido_eop[0] >= (especifico_peroxido_eop[0] - actual_value_especifico_peroxido_eop), name = 'diff_especifico_peroxido_eop positive segment')
m.addConstr(diff_especifico_peroxido_eop[0] >= -(especifico_peroxido_eop[0] - actual_value_especifico_peroxido_eop), name = 'diff_especifico_peroxido_eop negative segment')
m.addConstr(diff_especifico_peroxido_eop[0] <= delta_especifico_peroxido_eop, name = 'diff_especifico_peroxido_eop delta')


# especifico_soda_eop
m.addConstr(diff_especifico_soda_eop[0] >= (especifico_soda_eop[0] - actual_value_especifico_soda_eop), name = 'diff_especifico_soda_eop positive segment')
m.addConstr(diff_especifico_soda_eop[0] >= -(especifico_soda_eop[0] - actual_value_especifico_soda_eop), name = 'diff_especifico_soda_eop negative segment')
m.addConstr(diff_especifico_soda_eop[0] <= delta_especifico_soda_eop, name = 'diff_especifico_soda_eop delta')


###### model d1_brillo ######
# especifico_dioxido_d1
m.addConstr(diff_especifico_dioxido_d1[0] >= (especifico_dioxido_d1[0] - actual_value_especifico_dioxido_d1), name = 'diff_especifico_dioxido_d1 positive segment')
m.addConstr(diff_especifico_dioxido_d1[0] >= -(especifico_dioxido_d1[0] - actual_value_especifico_dioxido_d1), name = 'diff_especifico_dioxido_d1 negative segment')
m.addConstr(diff_especifico_dioxido_d1[0] <= delta_especifico_dioxido_d1, name = 'diff_especifico_dioxido_d1 delta')


# especifico_acido_d1
m.addConstr(diff_especifico_acido_d1[0] >= (especifico_acido_d1[0] - actual_value_especifico_acido_d1), name = 'diff_especifico_acido_d1 positive segment')
m.addConstr(diff_especifico_acido_d1[0] >= -(especifico_acido_d1[0] - actual_value_especifico_acido_d1), name = 'diff_especifico_acido_d1 negative segment')
m.addConstr(diff_especifico_acido_d1[0] <= delta_especifico_acido_d1, name = 'diff_especifico_acido_d1 delta')


###### model p_blancura ######
# especifico_soda_p
m.addConstr(diff_especifico_soda_p[0] >= (especifico_soda_p[0] - actual_value_especifico_soda_p), name = 'diff_especifico_soda_p positive segment')
m.addConstr(diff_especifico_soda_p[0] >= -(especifico_soda_p[0] - actual_value_especifico_soda_p), name = 'diff_especifico_soda_p negative segment')
m.addConstr(diff_especifico_soda_p[0] <= delta_especifico_soda_p, name = 'diff_especifico_soda_p delta')


# especifico_peroxido_p
m.addConstr(diff_especifico_peroxido_p[0] >= (especifico_peroxido_p[0] - actual_value_especifico_peroxido_p), name = 'diff_especifico_peroxido_p positive segment')
m.addConstr(diff_especifico_peroxido_p[0] >= -(especifico_peroxido_p[0] - actual_value_especifico_peroxido_p), name = 'diff_especifico_peroxido_p negative segment')
m.addConstr(diff_especifico_peroxido_p[0] <= delta_especifico_peroxido_p, name = 'diff_especifico_peroxido_p delta')


# especifico_acido_p
m.addConstr(diff_especifico_acido_p[0] >= (especifico_acido_p[0] - actual_value_especifico_acido_p), name = 'diff_especifico_acido_p positive segment')
m.addConstr(diff_especifico_acido_p[0] >= -(especifico_acido_p[0] - actual_value_especifico_acido_p), name = 'diff_especifico_acido_p negative segment')
m.addConstr(diff_especifico_acido_p[0] <= delta_especifico_acido_p, name = 'diff_especifico_acido_p delta')

<gurobi.Constr *Awaiting Model Update*>

In [31]:
m

<gurobi.Model Continuous instance Bleaching Optimization v2: 0 constrs, 0 vars, No parameter changes>

In [32]:
# update model
m.update()

In [33]:
m

<gurobi.Model Continuous instance Bleaching Optimization v2: 27 constrs, 21 vars, No parameter changes>

### 7. Add constraints that are machine learning models
To add constraints that have machine learning models it is necessary define a dataframe that are the instance of prediction (it has columns as gurobi decision variables) and then create the constraint in gurobi.

In this example, where each region has its own model, the dataframe instance also needs to be defined indidually. For the decision variable that are defined in the set "regions" it is important filter the dataframe instance with the correct element of the set region

**So, for each element in set region will be defined the instance dataframe and a constraint. Each region has it own model**Also, the instance has only one row, so now it is possible define a optimization model with set "time" and each row of the dataframe could be the instance of time t, t+1, t+2, etc


**IMPORTANT: LOGICALLY, FOR THIS EXAMPLE, TO DEFINE THE CONSTRAINTS OF ML MODELS, A FOR COULD HAVE BEEN MADE IN THE SET "REGIONS" BUT IT WAS NOT DONE CONSCIOUSLY THINKING OF AN EXAMPLE IN WHICH RESTRICTIONS HAVE TO BE DEFINED IN DIFFERENT SETS**

#### 7.1 d0eop_microkappa

In [34]:
######################## instance model d0eop_microkappa ########################

# create instance with controlables variables. sorted according the list of features. ES MUY IMPORTANTE QUE ESTÉ ORDENADO LAS VARIABLES DE DECUISIÓN DE ACUERDO A LA LISTA DE FEATURES
instance_controlables_d0eop_microkappa = pd.DataFrame([especifico_dioxido_d0, especifico_oxigeno_eop, especifico_peroxido_eop, especifico_soda_eop]).T
instance_controlables_d0eop_microkappa.columns = list_features_controlables_d0eop_microkappa # rename columns
instance_controlables_d0eop_microkappa.reset_index(inplace = True)
instance_controlables_d0eop_microkappa.drop(columns = 'index', inplace = True)

# append features controlables with no controlables
instance_d0eop_microkappa = pd.concat([instance_no_controlables_d0eop_microkappa, instance_controlables_d0eop_microkappa], axis = 1)
instance_d0eop_microkappa = instance_d0eop_microkappa[list_features_d0eop_microkappa] # sort features

# set index - optimization set
instance_d0eop_microkappa.index = index_bleaching

instance_d0eop_microkappa

,240AIT063A.PNT,240AIT063B.PNT,calc_prod_d0,240FY050.RO02,SSTRIPPING015,S276PER002,240AIC022.MEAS,240FY118B.RO01,240FY11PB.RO01,240FY107A.RO01
bleaching,6.349346,61.826925,3346.85825,<gurobi.Var especifico_dioxido_d0[bleaching]>,707.265937,11.516543,2.93846,<gurobi.Var especifico_oxigeno_eop[bleaching]>,<gurobi.Var especifico_peroxido_eop[bleaching]>,<gurobi.Var especifico_soda_eop[bleaching]>


In [35]:
###### load ml constraint ######
pred_constr_d0eop_microkappa = add_predictor_constr(gp_model = m, 
                                   predictor = model_d0eop_microkappa, 
                                   input_vars = instance_d0eop_microkappa, 
                                   output_vars = microkappa_d1,
                                   name = f'model_predict_d0eop_microkappa'
                                  )
pred_constr_d0eop_microkappa.print_stats()

Model for model_predict_d0eop_microkappa:
72 variables
1 constraints
66 quadratic constraints
Input has shape (1, 10)
Output has shape (1, 1)

Pipeline has 2 steps:

--------------------------------------------------------------------------------
Step            Output Shape    Variables              Constraints              
                                                Linear    Quadratic      General
poly_feat            (1, 66)           72            0           66            0

lin_reg               (1, 1)            0            1            0            0

--------------------------------------------------------------------------------


#### 7.2 d1_brillo

In [36]:
# (reemplazar "d0eop_microkappa" por "d1_brillo")
######################## instance model d1_brillo ########################

# create instance with controlables variables. sorted according the list of features. ES MUY IMPORTANTE QUE ESTÉ ORDENADO LAS VARIABLES DE DECUISIÓN DE ACUERDO A LA LISTA DE FEATURES
instance_controlables_d1_brillo = pd.DataFrame([especifico_dioxido_d1, especifico_acido_d1]).T # <---- change ---------<--------<--------
instance_controlables_d1_brillo.columns = list_features_controlables_d1_brillo # rename columns
instance_controlables_d1_brillo.reset_index(inplace = True)
instance_controlables_d1_brillo.drop(columns = 'index', inplace = True)

# create instance with target of previos model
instance_previos_target_d1_brillo = pd.DataFrame([microkappa_d1]).T # <---- change ---------<--------<--------
instance_previos_target_d1_brillo.columns  = list_target_d0eop_microkappa # rename columns
instance_previos_target_d1_brillo.reset_index(inplace = True)
instance_previos_target_d1_brillo.drop(columns = 'index', inplace = True)

# append features controlables with no controlables
instance_d1_brillo = pd.concat([instance_no_controlables_d1_brillo, instance_controlables_d1_brillo, instance_previos_target_d1_brillo], axis = 1)
instance_d1_brillo = instance_d1_brillo[list_features_d1_brillo] # sort features

# set index - optimization set
instance_d1_brillo.index = index_bleaching

instance_d1_brillo

,240AIT225B.PNT,240AIT225A.PNT,240FI108A.PNT,calc_prod_d1,240FY218.RO02,240TIT223.PNT,240FY210A.RO01
bleaching,85.051634,<gurobi.Var especifico_dioxido_d0[bleaching]>,169.45041,3347.664057,<gurobi.Var especifico_dioxido_d1[bleaching]>,77.212822,<gurobi.Var especifico_acido_d1[bleaching]>


In [37]:
###### load ml constraint ######
pred_constr_d1_brillo = add_predictor_constr(gp_model = m, 
                                   predictor = model_d1_brillo, 
                                   input_vars = instance_d1_brillo, 
                                   output_vars = brillo_entrada_p,
                                   name = f'model_predict_d1_brillo'
                                  )

pred_constr_d1_brillo.print_stats()

Model for model_predict_d1_brillo:
4 variables
1 constraints
Input has shape (1, 7)
Output has shape (1, 1)


#### 7.3 model p_blancura

In [38]:
# (reemplazar "d1_brillo" por "p_blancura")
######################## instance model p_blancura ########################

# create instance with controlables variables. sorted according the list of features. ES MUY IMPORTANTE QUE ESTÉ ORDENADO LAS VARIABLES DE DECUISIÓN DE ACUERDO A LA LISTA DE FEATURES
instance_controlables_p_blancura = pd.DataFrame([especifico_soda_p, especifico_peroxido_p, especifico_acido_p]).T # <---- change ---------<--------<--------
instance_controlables_p_blancura.columns = list_features_controlables_p_blancura # rename columns
instance_controlables_p_blancura.reset_index(inplace = True)
instance_controlables_p_blancura.drop(columns = 'index', inplace = True)

# create instance with target of previos model
instance_previos_target_p_blancura = pd.DataFrame([brillo_entrada_p]).T # <---- change ---------<--------<--------
instance_previos_target_p_blancura.columns  = list_target_d1_brillo # rename columns
instance_previos_target_p_blancura.reset_index(inplace = True)
instance_previos_target_p_blancura.drop(columns = 'index', inplace = True)

# append features controlables with no controlables
instance_p_blancura = pd.concat([instance_no_controlables_p_blancura, instance_controlables_p_blancura, instance_previos_target_p_blancura], axis = 1)
instance_p_blancura = instance_p_blancura[list_features_p_blancura] # sort features

# set index - optimization set
instance_p_blancura.index = index_bleaching

instance_p_blancura

,240AIT322B.PNT,calc_prod_p,240FY312.RO01,240FY397.RO01,240AIC324.MEAS,240FY430.RO01
bleaching,<gurobi.Var especifico_dioxido_d0[bleaching]>,3346.185561,<gurobi.Var especifico_soda_p[bleaching]>,<gurobi.Var especifico_peroxido_p[bleaching]>,10.382279,<gurobi.Var especifico_acido_p[bleaching]>


In [39]:
###### load ml constraint ######
pred_constr_p_blancura = add_predictor_constr(gp_model = m, 
                                   predictor = model_p_blancura, 
                                   input_vars = instance_p_blancura, 
                                   output_vars = blancura_salida_p,
                                   name = f'model_predict_p_blancura'
                                  )

pred_constr_p_blancura.print_stats()

Model for model_predict_p_blancura:
30 variables
1 constraints
28 quadratic constraints
Input has shape (1, 6)
Output has shape (1, 1)

Pipeline has 2 steps:

--------------------------------------------------------------------------------
Step            Output Shape    Variables              Constraints              
                                                Linear    Quadratic      General
poly_feat            (1, 28)           30            0           28            0

lin_reg               (1, 1)            0            1            0            0

--------------------------------------------------------------------------------


### 8. Define Objetive Function
The goal is to maximize the **net revenue**, which is the product of price and quantity, minus costs over all regions. This model assumes the purchase costs are fixed (since the amount $B$ is fixed) and are therefore not incorporated.

\begin{align} \textrm{maximize} &  \sum_{r}  (p_r * s_r - c_{waste} * u_r -
c^r_{transport} * x_r)& \end{align}

In [40]:
# ######################## define variable of costs of each stage ########################
costs_d0 = especifico_dioxido_d0*prices['dioxido'].values[0]
costs_eop = especifico_soda_eop*prices['soda'].values[0] + especifico_peroxido_eop*prices['peroxido'].values[0] + especifico_oxigeno_eop*prices['oxigeno'].values[0]
costs_d1 = especifico_acido_d1*prices['acido'].values[0] + especifico_dioxido_d1*prices['dioxido'].values[0]
costs_p = especifico_acido_p*prices['acido'].values[0] + especifico_soda_p*prices['soda'].values[0] + especifico_peroxido_p*prices['peroxido'].values[0]

In [41]:
costs_d0

bleaching    1.4 especifico_dioxido_d0[bleaching]
Name: especifico_dioxido_d0, dtype: object

In [42]:
costs_d0.sum()

<gurobi.LinExpr: 1.4 especifico_dioxido_d0[bleaching]>

In [43]:
######################## set objetive minimize costs ########################

# it is necesary define with .sum() to get a guroli linear expression
m.setObjective(costs_d0.sum()+ costs_eop.sum() + costs_d1.sum() + costs_p.sum(),
               gp.GRB.MAXIMIZE)

### 9. Solve optimization problem
The objective is **quadratic** since we take the product of price and the predicted sales, both of which are variables. Maximizing a quadratic
term is said to be **non-convex**, and we specify this by setting the value of the [Gurobi NonConvex
parameter](https://www.gurobi.com/documentation/10.0/refman/nonconvex.html) to be $2$.

#### 9.1 Solve optimization problem

In [44]:
# solve
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19043.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 30 rows, 127 columns and 149 nonzeros
Model fingerprint: 0x64439bf8
Model has 94 quadratic constraints
Coefficient statistics:
  Matrix range     [2e-09, 3e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [9e-02, 1e+00]
  Bounds range     [1e+00, 3e+03]
  RHS range        [5e-01, 4e+02]
  QRHS range       [1e+00, 1e+00]
Presolve removed 27 rows and 55 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 27 rows and 95 columns
Presolve time: 0.00s
Presolved: 75 rows, 32 columns, 219 nonzeros
Presolved model has 20 bilinear constraint(s)
Variable types: 32 continuous, 0 integer (0 binary)
Found heuristic solution: objective 31.1837048

Root relaxation: int

In [45]:
#### know the status of the model - 2 a optimal solution was founded
# docu: https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes
m.Status

2

#### 9.2 Save optimal values in a dataframe
To get the optimal values of decision variables it is neccesary call "var.gppd.X"

In [46]:
######## create a dataframe with set as index
solution = pd.DataFrame(index = index_bleaching)

######################## save optimal values - features of models (only the features) ########################

# model d0eop_microkappa
solution["especifico_dioxido_d0"] = especifico_dioxido_d0.gppd.X
solution["especifico_oxigeno_eop"] = especifico_oxigeno_eop.gppd.X
solution["especifico_peroxido_eop"] = especifico_peroxido_eop.gppd.X
solution["especifico_soda_eop"] = especifico_soda_eop.gppd.X

# model d1_brillo
solution["especifico_dioxido_d1"] = especifico_dioxido_d1.gppd.X
solution["especifico_acido_d1"] = especifico_acido_d1.gppd.X

# model p_blancura
solution["especifico_soda_p"] = especifico_soda_p.gppd.X
solution["especifico_peroxido_p"] = especifico_peroxido_p.gppd.X
solution["especifico_acido_p"] = especifico_acido_p.gppd.X


######################## save optimal values - targets of models (some targets are features of the model of the next step) ########################
solution["microkappa_d1"] = microkappa_d1.gppd.X  # model d0eop_microkappa
solution["brillo_entrada_p"] = brillo_entrada_p.gppd.X  # model d1_brillo
solution["blancura_salida_p"] = blancura_salida_p.gppd.X  # model p_blancura


######################## round values ########################
solution = solution.round(3)


######################## # get value objetive function ########################
opt_cost = m.ObjVal

In [47]:
# show value objetive function
print("\n The optimal net revenue: $%f K dolars" % opt_cost)


 The optimal net revenue: $31.183705 K dolars


In [48]:
# show value decision variables
solution

,especifico_dioxido_d0,especifico_oxigeno_eop,especifico_peroxido_eop,especifico_soda_eop,especifico_dioxido_d1,especifico_acido_d1,especifico_soda_p,especifico_peroxido_p,especifico_acido_p,microkappa_d1,brillo_entrada_p,blancura_salida_p
bleaching,6.96,1.778,5.9,9.84,3.89,2.93,3.65,1.78,3.01,1.9,90.267,90.915


We can also check the error in the estimate of the Gurobi solution for the regression model.

### 9.3 Show actual values

In [49]:
######## create a dataframe with set as index
actual_values = pd.DataFrame(index = index_bleaching)

######################## save optimal values - features of models (only the features) ########################

# model d0eop_microkappa
actual_values["especifico_dioxido_d0"] = actual_value_especifico_dioxido_d0
actual_values["especifico_oxigeno_eop"] = actual_value_especifico_oxigeno_eop
actual_values["especifico_peroxido_eop"] = actual_value_especifico_peroxido_eop
actual_values["especifico_soda_eop"] = actual_value_especifico_soda_eop

# model d1_brillo
actual_values["especifico_dioxido_d1"] = actual_value_especifico_dioxido_d1
actual_values["especifico_acido_d1"] = actual_value_especifico_acido_d1

# model p_blancura
actual_values["especifico_soda_p"] = actual_value_especifico_soda_p
actual_values["especifico_peroxido_p"] = actual_value_especifico_peroxido_p
actual_values["especifico_acido_p"] = actual_value_especifico_acido_p

In [50]:
actual_values

,especifico_dioxido_d0,especifico_oxigeno_eop,especifico_peroxido_eop,especifico_soda_eop,especifico_dioxido_d1,especifico_acido_d1,especifico_soda_p,especifico_peroxido_p,especifico_acido_p
bleaching,6.46,1.08,4.9,8.84,2.89,1.93,2.65,0.78,2.01


In [51]:
#### show deltas - ej especifico de oxigeno que da bien

In [52]:
m.addConstr(diff_especifico_oxigeno_eop[0] >= (especifico_oxigeno_eop[0] - actual_value_especifico_oxigeno_eop), name = 'diff_especifico_oxigeno_eop positive segment')
m.addConstr(diff_especifico_oxigeno_eop[0] >= -(especifico_oxigeno_eop[0] - actual_value_especifico_oxigeno_eop), name = 'diff_especifico_oxigeno_eop negative segment')
m.addConstr(diff_especifico_oxigeno_eop[0] <= delta_especifico_oxigeno_eop, name = 'diff_especifico_oxigeno_eop delta')

<gurobi.Constr *Awaiting Model Update*>

In [53]:
especifico_oxigeno_eop.gppd.X

bleaching    1.778422
Name: especifico_oxigeno_eop, dtype: float64

In [54]:
actual_value_especifico_oxigeno_eop

1.08

In [55]:
(especifico_oxigeno_eop.gppd.X - actual_value_especifico_oxigeno_eop)  # el valor de la diferencia es

bleaching    0.698422
Name: especifico_oxigeno_eop, dtype: float64

In [56]:
- (especifico_oxigeno_eop.gppd.X - actual_value_especifico_oxigeno_eop)

bleaching   -0.698422
Name: especifico_oxigeno_eop, dtype: float64

In [57]:
diff_especifico_oxigeno_eop.gppd.X

bleaching    1.0
Name: diff_especifico_oxigeno_eop, dtype: float64